In [14]:
import os, sys, argparse
import pandas as pd
import matplotlib
import backtrader as bt
#from strategies.GoldenCross import GoldenCross
from strategies.BuyHold import BuyHold

cerebro = bt.Cerebro()
cerebro.broker.setcash(100000)

Nifty_prices = pd.read_csv("../../Data/Nifty-1D.csv", index_col = "YYYYMMDD", parse_dates = True)

feed = bt.feeds.PandasData(dataname=Nifty_prices)
cerebro.adddata(feed)
cerebro.addstrategy(BuyHold)

cerebro.run()
cerebro.plot()

ModuleNotFoundError: No module named 'strategies.BuyHold'

In [30]:
import math
import backtrader as bt
#import matplotlib
import warnings
import datetime
from datetime import datetime
from matplotlib.dates import (HOURS_PER_DAY, MIN_PER_HOUR, SEC_PER_MIN,
                              MONTHS_PER_YEAR, DAYS_PER_WEEK,
                              SEC_PER_HOUR, SEC_PER_DAY,
                              num2date, rrulewrapper, YearLocator,
                              MicrosecondLocator)
#from matplotlib.dates

class GoldenCrossover(bt.Strategy):
    params = {('fast', 50), ('slow', 200), ('order_percentage', 0.95), ('ticker', 'Nifty-1D')}
    
    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print(txt, dt)
        
    def __init__(self):
        self.fast_moving_average = bt.indicators.SMA(self.data.close, period=self.params.fast, plotname='50 day moving average')
        self.slow_moving_average = bt.indicators.SMA(self.data.close, period=self.params.slow, plotname='200 day moving average')
        self.crossover = bt.indicators.CrossOver(self.fast_moving_average, self.slow_moving_average)
        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:  
            return       
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %  # we using %.2f for rounding the decimals 
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))
                self.buyprice = order.executed.price    # So buy price will become order.executed price and same as like order.executed comm
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))
            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:                     # Here if trade is not closed it will wait until it close, if it close it print operation profit & Gross etc
            return
        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
        
    def next(self):
        if self.position.size == 0:
            if self.crossover > 0:
                amount_to_invest = (self.params.order_percentage * self.broker.cash)
                self.size = math.floor(amount_to_invest/self.data.close)
                
                print("Buy {} shares of {} at {}".format(self.size, self.params.ticker, self.data.close[0]))
                self.buy(sizer=self.size)
        if self.position.size > 0:
            if self.crossover < 0:
                print("Sell {} shares of {} at {}".format(self.size, self.params.ticker, self.data.close[0]))
                self.close()
                
if __name__ == "__main__":
    cerebro = bt.Cerebro()
    cerebro.addstrategy(GoldenCrossover)
    data = bt.feeds.YahooFinanceCSVData(
        dataname="../../Data/TCS.NS.csv",
        fromdate=datetime(2003, 1, 1),
        todate=datetime(2010, 12, 31),
        reverse=False)

    cerebro.adddata(data)
    cerebro.broker.setcash(100000.0)
   
    cerebro.broker.setcommission(commission=0.001)    # Set the commission - 0.1% ... divide by 100 to remove the %
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    cerebro.run()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    cerebro.plot()
        

Starting Portfolio Value: 100000.00
Buy 1104 shares of Nifty-1D at 86.0
BUY EXECUTED, Price: 86.27, Cost: 86.27, Comm 0.09 2004-05-07
Sell 1104 shares of Nifty-1D at 118.64
SELL EXECUTED, Price: 119.17, Cost: 86.27, Comm 0.12 2005-06-16
OPERATION PROFIT, GROSS 32.90, NET 32.69 2005-06-16
Buy 767 shares of Nifty-1D at 123.78
BUY EXECUTED, Price: 126.23, Cost: 126.23, Comm 0.13 2005-07-19
Sell 767 shares of Nifty-1D at 220.57
SELL EXECUTED, Price: 220.81, Cost: 126.23, Comm 0.22 2007-07-18
OPERATION PROFIT, GROSS 94.58, NET 94.23 2007-07-18
Buy 686 shares of Nifty-1D at 138.58
BUY EXECUTED, Price: 132.43, Cost: 132.43, Comm 0.13 2009-06-01
Final Portfolio Value: 100470.35


ImportError: cannot import name 'warnings' from 'matplotlib.dates' (D:\user\lenovo\lib\site-packages\matplotlib\dates.py)